In [1]:
import polars as pl
import pyodbc
from datetime import date

In [2]:
import time
start_time = time.time()

### Definición de Rutas

In [3]:
cierre = "202505"
cierre_l = "May25"
cierre_d = date(2025,5,31)

# Carpetas
wd = "E:/Users/jhernandezr/DAR/garantias/reporte/fotos/"
wd_data_raw = wd + "data/raw/"
wd_data_external = wd + "data/external/"
wd_data_processed_dwh = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/dwh/"
wd_data_processed_fotos = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/fotos/"
wd_data_processed_curva_recup = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/processed/curva_recup/"
wd_data_validations = "E:/Users/jhernandezr/DAR/garantias/data_pipeline_garantias/data/validations/"

# Inputs
fl_catalogos = wd_data_external + f'Catálogos_{cierre_l}'
fl_recuperadas_detalle = wd_data_processed_dwh + f'parquet/Recupera_con_Pagos_Flujos_{cierre}.parquet'
fl_pagadas_detalle = wd_data_processed_dwh + f'parquet/Pagadas_Global_VF_{cierre}.parquet'

# Outputs
# fl_tabla_ult_pago = wd_data_processed_curva_recup
fl_curva_recup_vf = wd_data_processed_curva_recup + f"CURVA_RECUP_{cierre}_VF.parquet"
fl_pagadas_agrup_vf = wd_data_processed_curva_recup + f"PAGADAS_AGRUP_{cierre}_VF.parquet"
fl_pagadas_detalle_vf = wd_data_processed_curva_recup + f"PAGADAS_DETALLE_VF_{cierre}.parquet"
fl_recup_agrup_vf = wd_data_processed_curva_recup + f"RECUP_AGRUP_{cierre}_VF.parquet"
fl_tabla_sev_obs = wd_data_processed_curva_recup + f"TABLA_SEV_OBS_{cierre}.parquet"

fl_valida_pagos = wd_data_validations + f"Valida_Pagos_CR.csv"
fl_valida_pagos_fide = wd_data_validations + f"Valida_Pagos_FIDE.csv"
fl_valida_recup_rescat = wd_data_validations + f"Valida_Recup_Rescat.csv"
fl_valida_recup_rescat_fide = wd_data_validations + f"Valida_Recup_Rescat_FIDE.csv"

### Importa tablas

In [4]:
schema_recuperadas = {
    'Concatenar_Saldos': pl.String,
    'Fecha_Consulta': pl.Datetime,
    'Programa_Id': pl.Int32,
    'Tipo_Garantia_Id': pl.Int16,
    'Tipo_Cambio_Cierre': pl.Float32,
    'Programa_Original': pl.Int32,
    'Porcentaje_Garantizado': pl.Float32,
    'Monto _Credito_Mn': pl.Float64,
    'Fecha_Apertura': pl.Datetime,
    'Moneda_Id': pl.Int16,
    'Tipo_Credito_Id': pl.Int16,
    'Fecha_Garantia_Honrada': pl.Datetime,
    'TPRO_CLAVE': pl.Int32,
    'NR_R': pl.String,
    'Producto': pl.String,
    'AGRUPAMIENTO_ID': pl.Float64,
    'AGRUPAMIENTO': pl.String,
    'Intermediario_Id': pl.String,
    'Numero_Credito': pl.String,
    'Id': pl.Int16,
    'Monto': pl.Float64,
    'Interes': pl.Float64,
    'Moratorios': pl.Float64,
    'Descripcion': pl.String,
    'Estatus': pl.String,
    'Fecha_Registro': pl.Datetime,
    'Fecha': pl.Datetime,
    'Monto_Mn': pl.Float64,
    'Interes_Mn': pl.Float64,
    'Excedente': pl.Float64,
    'Excedente_Mn': pl.Float64,
    'Moratorios_Mn': pl.Float64,
    'Sub_Total_Mn': pl.Float64,
    'RazÃ³n Social (Intermediario)': pl.String,
    'Empresa / Acreditado (DescripciÃ³n)': pl.String,
    'RFC Empresa / Acreditado': pl.String,
    'TIPO_PERSONA': pl.String,
    'Recup_Rescat': pl.String,
    'MM_UDIS': pl.Int64,
    'ESQUEMA': pl.String,
    'CSG': pl.String,
    'CSF': pl.String,
    'Gastos_Juicio_Mn': pl.Float64,
    'Monto_Total_Mn': pl.Float64,
    'MPagado_Mn': pl.Float64,
    'Historico': pl.String,
    'Fecha Registro Alta': pl.Datetime}

schema_pagadas = {
    'Concatenado': pl.String,
    'Fecha_Consulta': pl.Datetime,
    'Intermediario_Id': pl.String,
    'Numero_Credito': pl.String,
    'Producto': pl.String,
    'Pago ID': pl.Int16,
    'RazÃ³n Social (Intermediario)': pl.String,
    'MIN_Fecha_Registro': pl.Datetime,
    'Fecha_Garantia_Honrada': pl.Datetime,
    'TPRO_CLAVE': pl.Float32,
    'Programa_Original': pl.Int32,
    'Programa_Id': pl.Int32,
    'Monto _Credito_Mn': pl.Float64,
    'Moneda_Id': pl.Int16,
    'Fecha de Apertura': pl.Datetime,
    'Tipo_Garantia_Id': pl.Int16,
    'TIPO_PERSONA': pl.String,
    'RFC Empresa / Acreditado': pl.String,
    'Monto_Desembolsado': pl.Float64,
    'Interes_Desembolso': pl.Float64,
    'Interes_Moratorios': pl.Float64,
    'Porcentaje_Garantizado': pl.Float32,
    'Tipo_Credito_Id': pl.Int16,
    'Estatus_Recuperacion': pl.String,
    'Empresa / Acreditado (DescripciÃ³n)': pl.String,
    'Fecha Registro Alta': pl.Datetime,
    'TC': pl.Float32,
    'AGRUPAMIENTO_ID': pl.Float32, # Revisar
    'AGRUPAMIENTO': pl.String,
    'ESQUEMA': pl.String,
    'SUBESQUEMA': pl.String,
    'CAMBIO': pl.Float64,
    'MM_UDIS': pl.Int16,
    'NR_R': pl.String,
    'CSG': pl.String,
    'CSF': pl.String,
    'Monto_Desembolso_Mn': pl.Float64,
    'Interes_Desembolso_Mn': pl.Float64,
    'Interes_Moratorios_Mn': pl.Float64,
    'Monto_Pagado_Mn': pl.Float64
}

In [5]:
# Importa Recuperadas
recuperadas_detalle = pl.read_parquet(fl_recuperadas_detalle)
""" recuperadas_detalle = (recuperadas_detalle
                        .filter(
                        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
                        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
                        (pl.col("Producto") != "GARANTIAS BANSEFI")
                        )) """

' recuperadas_detalle = (recuperadas_detalle\n                        .filter(\n                        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &\n                        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &\n                        (pl.col("Producto") != "GARANTIAS BANSEFI")\n                        )) '

In [6]:
# Importa Pagadas
pagadas_detalle = pl.read_parquet(fl_pagadas_detalle)
""" pagadas_detalle = (pagadas_detalle
                   .filter(
                        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
                        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
                        (pl.col("Producto") != "GARANTIAS BANSEFI")
                   ))
pagadas_detalle.shape """

' pagadas_detalle = (pagadas_detalle\n                   .filter(\n                        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &\n                        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &\n                        (pl.col("Producto") != "GARANTIAS BANSEFI")\n                   ))\npagadas_detalle.shape '

In [7]:
print(recuperadas_detalle.shape)
print(pagadas_detalle.shape)

(1566470, 47)
(514042, 40)


In [8]:
# Importa Catálogos
conn_str = r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=" + fl_catalogos + '.accdb'
conn = pyodbc.connect(conn_str)
estatus = pl.read_database(query='SELECT * FROM [ESTATUS]', connection=conn)
llave = pl.read_database(query='SELECT * FROM [LLAVE]', connection=conn, schema_overrides={'Programa Mezcla ID': pl.Int32})
conn.close()



In [9]:
# funciones Individuales Recuperadas
def gen_recup_previo(recuperadas, estatus):
    # Se realiza el Join
    df = recuperadas.join(
            estatus.select(["Estatus ID", "Estatus"]),
            left_on="Estatus",
            right_on="Estatus ID",
            how="left"
    )
    df = df.with_columns([
        (pl.col("Monto_Mn").fill_null(0) / 1_000_000).alias("CAPRECUP(MDP)"),
        (pl.col("Interes_Mn").fill_null(0) / 1_000_000).alias("INTRECUP(MDP)"),
        (pl.col("Moratorios_Mn").fill_null(0) / 1_000_000).alias("MORAT(MDP)"),
        (pl.col("Excedente_Mn").fill_null(0) / 1_000_000).alias("EXCEDENTE(MDP)"),
        (pl.col("Gastos_Juicio_Mn").fill_null(0) / 1_000_000).alias("GASTOSJUICIO(MDP)"),
        (pl.col("Sub_Total_Mn").fill_null(0) / 1_000_000).alias("SUBTOTAL (MDP)"),
        (pl.col("Monto_Total_Mn").fill_null(0) / 1_000_000).alias("MONTOTOTAL (MDP)"),
        (pl.when(pl.col("Estatus") == "E").then(pl.lit("E")).otherwise(pl.lit("NE")).alias("REEST")),
        pl.col("Estatus_right").alias("STATUS_RECUP"),
        pl.when(pl.col("Fecha") > pl.col("Fecha_Garantia_Honrada")).then(1).otherwise(0).alias("IND_ENTRA"),
        
        # MRECUP_TOT(MDP)
        pl.when(pl.col("Estatus_right") == "Recuperada").then(
            (
                pl.col("Monto_Mn").fill_null(0) +
                pl.col("Interes_Mn").fill_null(0) +
                pl.col("Moratorios_Mn").fill_null(0) +
                pl.col("Excedente_Mn").fill_null(0) -
                pl.col("Gastos_Juicio_Mn").fill_null(0)
            ) / 1_000_000
        ).otherwise(0).alias("MRECUP_TOT(MDP)"),
        
        # MRESCAT_TOT(MDP)
        pl.when(pl.col("Estatus_right") == "Rescatada").then(
            (
                pl.col("Monto_Mn").fill_null(0) +
                pl.col("Interes_Mn").fill_null(0) +
                pl.col("Moratorios_Mn").fill_null(0) +
                pl.col("Excedente_Mn").fill_null(0) -
                pl.col("Gastos_Juicio_Mn").fill_null(0)
            ) / 1_000_000
        ).otherwise(0).alias("MRESCAT_TOT(MDP)"),

        # Diferencia en meses entre fechas
        ((pl.col("Fecha").dt.year() - pl.col("Fecha_Garantia_Honrada").dt.year()) * 12 +
        (pl.col("Fecha").dt.month() - pl.col("Fecha_Garantia_Honrada").dt.month())).alias("DIFERENCIAS_MESES"),

        pl.col("Fecha_Garantia_Honrada").dt.year().alias("ANIO_PAGO"),
        (pl.col("Fecha_Garantia_Honrada").dt.year() * 100 + pl.col("Fecha_Garantia_Honrada").dt.month()).alias("ORDEN_PAGO"),

        # Concatenación de campos como string
        (pl.col("Numero_Credito").cast(str) + pl.col("Intermediario_Id").cast(str)).alias("CONCATENAR_SALDOS")
    ])

    return df


def gen_ciclos_rescate(df):


    df_filtered = df.filter(
        pl.col("Numero_Credito").is_not_null() & 
        pl.col("Razón Social (Intermediario)").is_not_null()
    )

    # Agrupar y sumar
    agg_df = (
        df_filtered
        .group_by(["Numero_Credito", "Razón Social (Intermediario)", "CONCATENAR_SALDOS"])
        .agg(
            (pl.col("MRESCAT_TOT(MDP)") * pl.col("IND_ENTRA")).sum().alias("MRESCAT_TOTAL")
        )
    )

    # Agregar columnas requeridas
    final_df = (
        agg_df.with_columns([
            pl.col("Numero_Credito").alias("Prestamo"),
            pl.col("Razón Social (Intermediario)").alias("INTERMEDIARIO"),
            pl.when(pl.col("MRESCAT_TOTAL") == 0).then(1).otherwise(0).alias("CICLOS_RESCAT")
        ])
        .select([
            "CONCATENAR_SALDOS", 
            "Prestamo", 
            "INTERMEDIARIO", 
            "MRESCAT_TOTAL", 
            "CICLOS_RESCAT"
        ])
    )
    return final_df



In [10]:
# Funciones Individuales Pagadas
def gen_recup_previo_cup(pagadas_con_llave, ult_pago):
    df_joined = pagadas_con_llave.join(
        ult_pago,
        on=["Intermediario_Id", "Numero_Credito"],
        how="left"
    )

    # Filtrar donde 'Pago ID' == 'ULT_PAGO' y ULT_PAGO no es null
    df_result = (
        df_joined.filter(
            (pl.col("Pago ID") == pl.col("ULT_PAGO")) &
            (pl.col("ULT_PAGO").is_not_null())
        )
    )
    return df_result



In [11]:
def gen_curva_recup(recuperadas_detalle, llave, estatus):
    """"
    Descripción:
    - Toma recuperadas_detalle, le pega la variable LLAVE_FINAL del catálogo llave y genera recuperadas_con_llave
    - Genera recup_previo a partir de recuperadas_con_llave y el catálogo de estatus
    - Genera ciclos_rescate a partir de recup_previo
    - Toma recup previo y le pega la variable CICLOS_RESCAT usando CONCATENAR_SALDOS
    """
    # Recuperadas_Detalle_cLlave
    recuperadas_con_llave = recuperadas_detalle.join(
        llave.select([
                "Programa Mezcla ID",
                "Intermediario_Id",
                "LLAVE_FINAL"]),
            left_on=["TPRO_CLAVE", "Intermediario_Id"],
            right_on=["Programa Mezcla ID", "Intermediario_Id"],
            how="left")
    
    recup_previo = gen_recup_previo(recuperadas_con_llave, estatus)
    ciclos_rescate = gen_ciclos_rescate(recup_previo)

    # CURVA_RECUP_202504
    curva_recup = recup_previo.join(
        ciclos_rescate.select(["CONCATENAR_SALDOS", "CICLOS_RESCAT"]),
        on="CONCATENAR_SALDOS",
        how="left"
    )

    
    return ciclos_rescate, curva_recup

ciclos_rescate, curva_recup = gen_curva_recup(recuperadas_detalle, llave, estatus)

In [12]:
recuperadas_detalle['CSF'].value_counts()

CSF,count
str,u32
"""SF""",96192
"""CF""",1470278


In [13]:
def gen_pagadas_detalle_vf(pagadas_detalle, ciclos_rescate, llave):

    
    pagadas_con_llave = pagadas_detalle.join(
        llave.select([
                "Programa Mezcla ID",
                "Intermediario_Id",
                "LLAVE_FINAL"]),
            left_on=["TPRO_CLAVE", "Intermediario_Id"],
            right_on=["Programa Mezcla ID", "Intermediario_Id"],
            how="left")
    
    # TABLA_ULT_PAGO
    tabla_ult_pago = (
        pagadas_con_llave.group_by(["Intermediario_Id", "Numero_Credito"])
        .agg(pl.col("Pago ID").max().alias("ULT_PAGO"))
    )

    recup_previo_cup = gen_recup_previo_cup(pagadas_con_llave, tabla_ult_pago)

    pagadas_previo = (
        recup_previo_cup.with_columns([
            (pl.col("Monto_Pagado_Mn").fill_null(0) / 1_000_000).alias("MPAGADO (MDP)"),
            pl.col("Fecha_Garantia_Honrada").dt.year().alias("ANIO_PAGO"),
            (
                pl.col("Fecha_Garantia_Honrada").dt.year() * 100 +
                pl.col("Fecha_Garantia_Honrada").dt.month()
            ).alias("ORDEN_PAGO"),
            (pl.col("Numero_Credito").cast(str) + pl.col("Intermediario_Id").cast(str)).alias("CONCATENAR_SALDOS")
        ])
    )

    pagadas_detalle_vf = pagadas_previo.join(
        ciclos_rescate.select(["CONCATENAR_SALDOS", "CICLOS_RESCAT"]),
        on="CONCATENAR_SALDOS",
        how="left"
    )
    
    return pagadas_detalle_vf

pagadas_detalle_vf = gen_pagadas_detalle_vf(pagadas_detalle, ciclos_rescate, llave)

In [14]:
def gen_curva_recup_vf(pagadas_detalle_vf, curva_recup):
    # CURVA_RECUP_202504_VF
    curva_recup_vf = curva_recup.join(
        pagadas_detalle_vf.select(["CONCATENAR_SALDOS", "Monto_Pagado_Mn"]),
        on="CONCATENAR_SALDOS",
        how="left"
    ).with_columns(
        (pl.col("Monto_Pagado_Mn") / 1_000_000).alias("MPAGADO (MDP)")
    )
    return curva_recup_vf

curva_recup_vf = gen_curva_recup_vf(pagadas_detalle_vf, curva_recup)


In [15]:
def gen_recup_agrup(curva_recup):

    curva_recup = curva_recup.with_columns([
        pl.col("Fecha_Registro").dt.year().alias("Anio_REG_RECUP"),
        pl.col("Fecha_Registro").dt.month().alias("Mes_REG_RECUP")
    ])

    # Agrupar y sumar
    # RECUP_AGRUP_202504_inter    
    recup_agrup_inter = (
        curva_recup
        .group_by([
            "IND_ENTRA", "MM_UDIS", "Producto", "NR_R", "LLAVE_FINAL", "TPRO_CLAVE",
            "Razón Social (Intermediario)", "CSG", "CSF", "AGRUPAMIENTO", "AGRUPAMIENTO_ID",
            "Intermediario_Id", "CICLOS_RESCAT", "DIFERENCIAS_MESES", "Tipo_Garantia_Id",
            "ANIO_PAGO", "REEST", "Anio_REG_RECUP", "ESQUEMA", "ORDEN_PAGO"
        ])
        .agg([
            pl.col("MRECUP_TOT(MDP)").sum().alias("MRECUP_TOT"),
            pl.col("MRESCAT_TOT(MDP)").sum().alias("MRESCAT_TOT")
        ])
        .rename({
            "Producto": "TAXONOMIA",
            "Tipo_Garantia_Id": "TIPGAR_CLAVE",
            "ANIO_PAGO": "ANIO(PAGO)",
            "Razón Social (Intermediario)": "BANCO"
        })
    )

    # RECUP_AGRUP_202504_Completa
    recup_agrup_completa = recup_agrup_inter.with_columns([
        pl.when(pl.col("ESQUEMA") == "PP")
        .then(pl.lit("Pari Passu"))
        .when(pl.col("ESQUEMA").is_null())
        .then(pl.lit("Pari Passu"))
        .otherwise(pl.lit("1P"))
        .alias("ESQUEMA_VF"),

        pl.when(pl.col("ANIO(PAGO)") <= 2006)
        .then(2006)
        .otherwise(pl.col("ANIO(PAGO)"))
        .alias("ANIO(PAGO)_New")
    ])

    return recup_agrup_completa

recup_agrup = gen_recup_agrup(curva_recup)

In [16]:
def pagadas_agrup(pagadas_detalle_vf):
    # PAGADAS_AGRUP_202504_inter
    pagadas_agrup_inter = (
        pagadas_detalle_vf.group_by([
            "MM_UDIS", "Producto", "NR_R", "LLAVE_FINAL", "TPRO_CLAVE",
            "Razón Social (Intermediario)", "CSG", "CSF", "AGRUPAMIENTO",
            "AGRUPAMIENTO_ID", "Intermediario_Id", "CICLOS_RESCAT",
            "Tipo_Garantia_Id", "ANIO_PAGO", "ESQUEMA", "ORDEN_PAGO"
        ])
        .agg([
            pl.col("MPAGADO (MDP)").sum().alias("MPAGADO")
        ])
        .rename({
            "Producto": "TAXONOMIA",
            "Razón Social (Intermediario)": "BANCO",
            "Tipo_Garantia_Id": "TIPGAR_CLAVE",
            "ANIO_PAGO": "ANIO(PAGO)"
        })
    )

    # PAGADAS_AGRUP_202504_Completa
    pagadas_agrup_completa = pagadas_agrup_inter.with_columns([
        # CICLOS_RESCAT_VF: 1 si es nulo, 0 si es 0, 1 si no es 0
        pl.when(pl.col("CICLOS_RESCAT").is_null())
            .then(1)
            .when(pl.col("CICLOS_RESCAT") == 0)
            .then(0)
            .otherwise(1)
            .alias("CICLOS_RESCAT_VF"),
        # ESQUEMA_VF: 'Pari Passu' si ESQUEMA es 'PP' o null, sino '1P'
        pl.when(pl.col("ESQUEMA") == 'PP')
            .then(pl.lit("Pari Passu"))
            .when(pl.col("ESQUEMA").is_null())
            .then(pl.lit("Pari Passu"))
            .otherwise(pl.lit("1P"))
            .alias("ESQUEMA_VF"),
        # ANIO(PAGO)_New: si ANIO(PAGO) <= 2006, entonces 2006
        pl.when(pl.col("ANIO(PAGO)") <= 2006)
            .then(2006)
            .otherwise(pl.col("ANIO(PAGO)"))
            .alias("ANIO(PAGO)_New")
    ])
    pagadas_agrup_completa

    # PAGADAS_AGRUP_202504
    pagadas_agrup = pagadas_agrup_completa.select([
        "MM_UDIS",
        "TAXONOMIA",
        "NR_R",
        "BANCO",
        "CSG",
        "CSF",
        "AGRUPAMIENTO",
        "AGRUPAMIENTO_ID",
        pl.col("CICLOS_RESCAT_VF").alias("CICLOS_RESCAT"),
        "TIPGAR_CLAVE",
        "ANIO(PAGO)",
        "MPAGADO",
        pl.col("ESQUEMA_VF").alias("ESQUEMA"),
        "ANIO(PAGO)_New",
        "Intermediario_Id",
        "TPRO_CLAVE",
        "ORDEN_PAGO"
    ])

    return pagadas_agrup

pagadas_agrup = pagadas_agrup(pagadas_detalle_vf)

In [17]:
# print(pagadas_detalle_vf['CICLOS_RESCAT'].value_counts().sort("CICLOS_RESCAT"))

In [18]:
def gen_tabla_sev_obs(recup_agrup, pagadas_agrup):
    tabla_sev_obs = recup_agrup.filter(
        (pl.col("CICLOS_RESCAT") == 1) &
        (pl.col("ANIO(PAGO)") <= 2018) &
        (pl.col("IND_ENTRA") == 1)
    ).select([   # Seleccionar y transformar columnas
        pl.col("MM_UDIS"),
        pl.col("TAXONOMIA"),
        pl.col("NR_R"),
        pl.col("BANCO"),
        pl.col("CSG"),
        pl.col("AGRUPAMIENTO"),
        pl.col("AGRUPAMIENTO_ID"),
        pl.col("TIPGAR_CLAVE"),
        pl.col("ESQUEMA"),
        pl.col("TPRO_CLAVE"),
        pl.col("ANIO(PAGO)_New"),
        pl.lit("Recup").alias("CONCEPTO"),
        pl.when(pl.col("Anio_REG_RECUP") <= 2018)
        .then(pl.lit("Inicio"))
        .otherwise(pl.lit("Acum"))
        .alias("PERIODO"),
        pl.col("MRECUP_TOT").alias("MONTO")
    ])

    tabla_sev_obs_temp = pagadas_agrup.filter(
        (pl.col("CICLOS_RESCAT") == 1) &
        (pl.col("ANIO(PAGO)") <= 2018)
    ).select([    # Seleccionar y transformar columnas
        pl.col("MM_UDIS"),
        pl.col("TAXONOMIA"),
        pl.col("NR_R"),
        pl.col("BANCO"),
        pl.col("CSG"),
        pl.col("AGRUPAMIENTO"),
        pl.col("AGRUPAMIENTO_ID"),
        pl.col("TIPGAR_CLAVE"),
        pl.col("ESQUEMA"),
        pl.col("TPRO_CLAVE"),
        pl.col("ANIO(PAGO)_New"),
        pl.lit("Pago").alias("CONCEPTO"),
        pl.lit("Inicio").alias("PERIODO"),
        pl.col("MPAGADO").alias("MONTO")
    ])

    tabla_sev_obs = pl.concat([tabla_sev_obs, tabla_sev_obs_temp.cast({'TIPGAR_CLAVE': pl.Int32})])
    
    return tabla_sev_obs


tabla_sev_obs = gen_tabla_sev_obs(recup_agrup, pagadas_agrup)

In [19]:
# Salidas Pagos
excluir_taxonomia = [
    "GARANTIAS BANCOMEXT",
    "GARANTIAS SHF/LI FINANCIERO",
    "GARANTIAS SHF-LI FINANCIERO",
    "GARANTIAS BANSEFI"
]

# PAGADAS_AGRUP_202504_VF
pagadas_agrup_vf = pagadas_agrup.filter(~pl.col("TAXONOMIA").is_in(excluir_taxonomia))

valida_pagos = (
    pagadas_agrup_vf
    .group_by("TAXONOMIA")
    .agg(pl.col("MPAGADO").sum().alias("S_MPagado"))
)

valida_pagos_fide = (
    pagadas_agrup_vf
    .filter(pl.col("AGRUPAMIENTO") == "FIDE")
    .group_by("TAXONOMIA")
    .agg(pl.col("MPAGADO").sum().alias("S_MPagado"))
)




In [20]:
# Salidas Recuperaciones
# Definir lista de TAXONOMIA a excluir
excluir_taxonomia = [
    "GARANTIAS BANCOMEXT",
    "GARANTIAS SHF/LI FINANCIERO",
    "GARANTIAS SHF-LI FINANCIERO",
    "GARANTIAS BANSEFI"
]

# RECUP_AGRUP_202504_VF
recup_agrup_vf = recup_agrup.filter(~pl.col("TAXONOMIA").is_in(excluir_taxonomia))

valida_recup_rescat = (
    recup_agrup_vf
    .filter(pl.col("IND_ENTRA") == 1)
    .group_by("TAXONOMIA")
    .agg([
        pl.col("MRECUP_TOT").sum().alias("S_MRcup"),
        pl.col("MRESCAT_TOT").sum().alias("S_MRescat")
    ])
)

valida_recup_rescat_fide = (
    recup_agrup_vf
    .filter((pl.col("AGRUPAMIENTO") == "FIDE") & (pl.col("IND_ENTRA") == 1))
    .group_by("TAXONOMIA")
    .agg([
        pl.col("MRECUP_TOT").sum().alias("S_MRcup"),
        pl.col("MRESCAT_TOT").sum().alias("S_MRescat")
    ])
)


In [21]:
valida_recup_rescat_fide.sort('TAXONOMIA')

TAXONOMIA,S_MRcup,S_MRescat
str,f64,f64
"""GARANTIA MICROCREDITO""",192.302915,14.544884


### Export

In [22]:
curva_recup_vf.write_parquet(fl_curva_recup_vf)
pagadas_agrup_vf.write_parquet(fl_pagadas_agrup_vf)
pagadas_detalle_vf.write_parquet(fl_pagadas_detalle_vf)
recup_agrup_vf.write_parquet(fl_recup_agrup_vf)
tabla_sev_obs.write_parquet(fl_tabla_sev_obs)

valida_pagos.write_csv(fl_valida_pagos, float_precision=10, float_scientific=False)
valida_pagos_fide.write_csv(fl_valida_pagos_fide, float_precision=10, float_scientific=False)
valida_recup_rescat.write_csv(fl_valida_recup_rescat, float_precision=10, float_scientific=False)
valida_recup_rescat_fide.write_csv(fl_valida_recup_rescat_fide, float_precision=10, float_scientific=False)


In [23]:
end_time = time.time()
end_time - start_time

8.262984275817871